In [35]:
%%bash
#https://strimzi.io/quickstarts/
#https://snourian.com/kafka-kubernetes-strimzi-part-3-monitoring-strimzi-kafka-with-prometheus-grafana/
#https://github.com/nrsina/strimzi-kafka-tutorial/tree/master/kafka-deployment
#https://strimzi.io/docs/operators/latest/deploying.html#proc-metrics-grafana-dashboard-str
# Kubernetes Cluster
KUBECONFIG=../CLUSTERS/mac.txt
#kubectl config view
#kubectl get nodes
#wget https://strimzi.io/examples/latest/kafka/kafka-persistent-single.yaml
#kubectl create namespace kafka
#kubectl create -f 'https://strimzi.io/install/latest?namespace=kafka' -n kafka
#kubectl delete -f kafka-persistent-single.yaml -n kafka 
kubectl create -f my-cluster-metrics.yaml -n kafka 

#kubectl wait kafka/my-cluster-metrics --for=condition=Ready --timeout=300s -n kafka 


kafka.kafka.strimzi.io/my-cluster-metrics created
configmap/kafka-metrics created


In [ ]:
%%bash
# Send
kubectl -n kafka run kafka-producer -ti --image=quay.io/strimzi/kafka:0.25.0-kafka-2.8.0 --rm=true --restart=Never -- bin/kafka-console-producer.sh --broker-list my-cluster-kafka-bootstrap:9092 --topic my-topic
# Receive
kubectl -n kafka run kafka-consumer -ti --image=quay.io/strimzi/kafka:0.25.0-kafka-2.8.0 --rm=true --restart=Never -- bin/kafka-console-consumer.sh --bootstrap-server my-cluster-kafka-bootstrap:9092 --topic my-topic --from-beginning

# Monitoring

In [67]:
%%bash
# https://snourian.com/kafka-kubernetes-strimzi-part-3-monitoring-strimzi-kafka-with-prometheus-grafana/
# Monitoring https://github.com/prometheus-community/helm-charts/blob/main/charts/kube-prometheus-stack/values.yaml
#helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
#https://strimzi.io/docs/operators/latest/deploying.html#proc-metrics-grafana-dashboard-str
#helm repo update
#helm install monitoring prometheus-community/kube-prometheus-stack -n monitoring --create-namespace

#kubectl delete secret  additional-scrape-configs -n monitoring

#curl -s https://raw.githubusercontent.com/coreos/prometheus-operator/master/bundle.yaml | sed -e '/[[:space:]]*namespace: [a-zA-Z0-9-]*$/s/namespace:[[:space:]]*[a-zA-Z0-9-]*$/namespace: monitoring/' > prometheus-operator-deployment.yaml
#kubectl apply -f prometheus-operator-deployment.yaml

#sed -i 's/namespace: .*/namespace: monitoring/' prometheus.yaml

kubectl apply -f prometheus-additional.yaml -n monitoring
kubectl apply -f strimzi-pod-monitor.yaml -n monitoring
kubectl apply -f prometheus-rules.yaml -n monitoring
kubectl apply -f prometheus.yaml -n monitoring
kubectl apply -f grafana.yaml -n monitoring


secret/additional-scrape-configs configured
podmonitor.monitoring.coreos.com/cluster-operator-metrics unchanged
podmonitor.monitoring.coreos.com/entity-operator-metrics unchanged
podmonitor.monitoring.coreos.com/bridge-metrics unchanged
podmonitor.monitoring.coreos.com/kafka-resources-metrics unchanged
prometheusrule.monitoring.coreos.com/prometheus-k8s-rules unchanged
clusterrole.rbac.authorization.k8s.io/prometheus-server unchanged
serviceaccount/prometheus-server unchanged
clusterrolebinding.rbac.authorization.k8s.io/prometheus-server unchanged
prometheus.monitoring.coreos.com/prometheus unchanged
deployment.apps/grafana created
service/grafana created


In [ ]:
# Grafana Dashboards
https://grafana.com/grafana/dashboards/10000
https://github.com/pivotal-cf/charts-grafana
# Disk
https://grafana.com/grafana/dashboards/13646
https://grafana.com/grafana/dashboards/12740

# Performance
https://medium.com/metrosystemsro/apache-kafka-how-to-test-performance-for-clients-configured-with-ssl-encryption-3356d3a0d52b